# 13 NetCDF and `xarray`

In this lesson, we will get acquainted with a popuar format for working with multidimensional datasets called NetCDF and the Python package `xarray` which is based on NetCDF. 


In [1]:
# Import packages
import os              
import numpy as np
import pandas as pd
import xarray as xr   

### Variable values

The underlying data in the `xarray.DataArray` is a `numpy.array` that holds the variable values. 

In [2]:
# Values of a single variable at each point of the coords 
temp_data = np.array([np.zeros((5,5)), 
                      np.ones((5,5)), 
                      np.ones((5,5))*2]).astype(int)
temp_data

array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]]])

### Dimensions and Coordinates

To specify the dimensions of our upcoming `xarray.DataArray`, we must examine how we've constructed the `numpy.array` holding the temperature data. 
The first dimension is time, the second is latitude, and longitude the third. 

From our exercises, we can also see that the coordinates (values of each dimension) are:

- time coordinates are 2022-09-01, 2022-09-02, 2022-09-03
- latitude coordinates are 70, 60, 50, 40, 30 (notice decreasing order)
- longitude coordinates are 60, 70, 80, 90, 100 (notice increasing order)

We add the dimensions as a tuple of strings and coordinates as a dictionary:

In [3]:
# Names of the dimensions in the required order
dims = ('time', 'lat', 'lon')
# Create coordinates to use for indexing along each dimension 
coords = {'time':pd.date_range("2022-09-01", "2022-09-03"),
          'lat':np.arange(70, 20, -10),
          'lon':np.arange(60, 110, 10)}  

#### Attributes

Next, we add the attributes (metadata) for our temperature data as a dictionary:

In [4]:
# Attributes (metadata) of the data array 
attrs = { 'title':'temperature across weather stations',
          'standard_name':'air_temperature',
          'units':'degree_c'}

#### Putting it all together

Finally, we put all these pieces together (data, dimensions, coordinates, and attributes) to create an `xarray.DataArray`:

In [5]:
# Initialize xarray.DataArray
temp = xr.DataArray(data = temp_data, 
                    dims = dims,
                    coords = coords,
                    attrs = attrs)
temp

<xarray.DataArray (time: 3, lat: 5, lon: 5)> Size: 600B
array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]]])
Coordinates:
  * time     (time) datetime64[ns] 24B 2022-09-01 2022-09-02 2022-09-03
  * lat      (lat) int64 40B 70 60 50 40 30
  * lon      (lon) int64 40B 60 70 80 90 100
Attributes:
    title:          temperature across weather stations
    standard_name:  air_temperature
    units:          degree_c

We can also update the variable’s attributes after creating the object. 
Notice that each of the coordinates is also an `xarray.DataArray`, so we can add attributes to them.

In [6]:
# Update attributes
temp.attrs['description'] = 'Simple example of an xarray.DataArray'

# Add attributes to coordinates 
temp.time.attrs = {'description':'date of measurement'}

temp.lat.attrs['standard_name']= 'grid_latitude'
temp.lat.attrs['units'] = 'degree_N'

temp.lon.attrs['standard_name']= 'grid_longitude'
temp.lon.attrs['units'] = 'degree_E'
temp

<xarray.DataArray (time: 3, lat: 5, lon: 5)> Size: 600B
array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]]])
Coordinates:
  * time     (time) datetime64[ns] 24B 2022-09-01 2022-09-02 2022-09-03
  * lat      (lat) int64 40B 70 60 50 40 30
  * lon      (lon) int64 40B 60 70 80 90 100
Attributes:
    title:          temperature across weather stations
    standard_name:  air_temperature
    units:          degree_c
    description:    Simple example of an xarray.DataArray

## Subsetting

To select data from an `xarray.DataArray` we need to specify the subsets we want along each dimension. We can specify the data we need from each dimension either by looking up the dimension by its position or by looking up each dimension by its name.

By Position 
- by interger `[]`
- by label `.loc[]`

By Name 
- by interger `.isel()`
- by label `.sel()`

In [7]:
# Access dimensions by position, then use integers for indexing
temp[0,3,2]

<xarray.DataArray ()> Size: 8B
array(0)
Coordinates:
    time     datetime64[ns] 8B 2022-09-01
    lat      int64 8B 40
    lon      int64 8B 80
Attributes:
    title:          temperature across weather stations
    standard_name:  air_temperature
    units:          degree_c
    description:    Simple example of an xarray.DataArray

In [8]:
# Access dimensions by position, then use labels for indexing
temp.loc['2022-09-01', 40, 80]

<xarray.DataArray ()> Size: 8B
array(0)
Coordinates:
    time     datetime64[ns] 8B 2022-09-01
    lat      int64 8B 40
    lon      int64 8B 80
Attributes:
    title:          temperature across weather stations
    standard_name:  air_temperature
    units:          degree_c
    description:    Simple example of an xarray.DataArray

In [9]:
# Acess dimensions by name, then use integers for indexing
temp.isel(time=0, lon=2, lat=3)

<xarray.DataArray ()> Size: 8B
array(0)
Coordinates:
    time     datetime64[ns] 8B 2022-09-01
    lat      int64 8B 40
    lon      int64 8B 80
Attributes:
    title:          temperature across weather stations
    standard_name:  air_temperature
    units:          degree_c
    description:    Simple example of an xarray.DataArray

In [10]:
# Access dimensions by name, then use labels for indexing
temp.sel(time='2022-09-01', lat=40, lon=80)

<xarray.DataArray ()> Size: 8B
array(0)
Coordinates:
    time     datetime64[ns] 8B 2022-09-01
    lat      int64 8B 40
    lon      int64 8B 80
Attributes:
    title:          temperature across weather stations
    standard_name:  air_temperature
    units:          degree_c
    description:    Simple example of an xarray.DataArray

Notice that the result of this indexing is a 1x1 xarray.DataArray. This is because operations on an xarray.DataArray always return another xarray.DataArray. In particular, operations returning scalar values will also produce xarray objects, so we need to cast them as numbers manually using the xarray.DataArray `item()` method:

In [11]:
temp.sel(time='2022-09-01', lat=40, lon=80).item()

0

## Reduction

xarray has implemented several methods to reduce an xarray.DataArray along any number of dimensions. For example, we can calculate the average temperature at each weather station over time and obtain a new xarray.DataArray.

In [12]:
avg_temp = temp.mean(dim = 'time') 

avg_temp.attrs = {'title':'Average temperature over three days'}
avg_temp

<xarray.DataArray (lat: 5, lon: 5)> Size: 200B
array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])
Coordinates:
  * lat      (lat) int64 40B 70 60 50 40 30
  * lon      (lon) int64 40B 60 70 80 90 100
Attributes:
    title:    Average temperature over three days

## xarray.DataSet

An `xarray.DataSet` resembles an in-memory representation of a NetCDF file and consists of multiple variables (each being an `xarray.DataArray`), with dimensions, coordinates, and attributes, forming a self-describing dataset. Attributes can be specific to each variable, each dimension, or they can describe the whole dataset. The variables in an `xarray.DataSet` can have the same dimensions, share some dimensions, or have no dimensions in common.

### Create an xarray.DataSet

Following our previous example, we can create an xarray.DataSet by combining the temperature data with the average temperature data. We also add some attributes that now describe the whole dataset, not only each variable.

In [13]:
# Make dictionaries with variables and attributes
data_vars = {'avg_temp': avg_temp,
             'temp': temp
             }

attrs = {'title':'Temperature data at weather stations: daily and and average',
         'description':'Simple example of an xarray.Dataset'
         }

# Create xarray.Dataset
temp_dataset = xr.Dataset( data_vars = data_vars,
                           attrs = attrs
                           )

Take some time to click through the data viewer and read through the variables and metadata in the dataset. Notice the following:

- `temp_dataset` is a dataset with three dimensions (time, latitude, and longitude),

- `temp` is a variable that uses all three dimensions in the dataset, and

- `aveg_temp` is a variable that only uses two dimensions (latitude and longitude).

In [14]:
temp_dataset

<xarray.Dataset> Size: 904B
Dimensions:   (lat: 5, lon: 5, time: 3)
Coordinates:
  * lat       (lat) int64 40B 70 60 50 40 30
  * lon       (lon) int64 40B 60 70 80 90 100
  * time      (time) datetime64[ns] 24B 2022-09-01 2022-09-02 2022-09-03
Data variables:
    avg_temp  (lat, lon) float64 200B 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    temp      (time, lat, lon) int64 600B 0 0 0 0 0 0 0 0 0 ... 2 2 2 2 2 2 2 2
Attributes:
    title:        Temperature data at weather stations: daily and and average
    description:  Simple example of an xarray.Dataset

## Save and reopen

Finally, we want to save our dataset as a NetCDF file. To do this, specify the file path and use the .nc extension for the file name. Then save the dataset using the `to_netcdf` method with your file path. Opening NetCDF is similarly straightforward using `xarray.open_dataset()`.

In [15]:
# Save xarray.DataSet as NetCDF file
temp_dataset.to_netcdf('temp_dataset.nc')

# Import NetCDF file
check = xr.open_dataset('temp_dataset.nc')
check

<xarray.Dataset> Size: 564B
Dimensions:   (lat: 5, lon: 5, time: 3)
Coordinates:
  * lat       (lat) int32 20B 70 60 50 40 30
  * lon       (lon) int32 20B 60 70 80 90 100
  * time      (time) datetime64[ns] 24B 2022-09-01 2022-09-02 2022-09-03
Data variables:
    avg_temp  (lat, lon) float64 200B ...
    temp      (time, lat, lon) int32 300B ...
Attributes:
    title:        Temperature data at weather stations: daily and and average
    description:  Simple example of an xarray.Dataset